In [19]:
import pandas as pd

TARGET_COLS = ['toxic',
               'severe_toxic',
               'obscene',
               'threat',
               'insult',
               'identity_hate']

train_df = pd.read_csv('../data/train.csv').sort_values(by='id')
bert_preds = pd.read_csv('../data/preds_bert.csv').sort_values(by='id')
gpt_preds = pd.read_csv('../data/preds_gpt.csv').sort_values(by='id')
bigru_fasttext_preds = pd.read_csv('../data/preds_bigru_fasttext_base.csv').sort_values(by='id')
bigru_elmo_preds = pd.read_csv('../data/preds_bigru_elmo_base.csv').sort_values(by='id')


In [20]:
# Confirm that the prediction and train set ids are one-to-one 
assert set(train_df.id) == set(bert_preds.id)
assert set(train_df.id) == set(gpt_preds.id)
assert set(train_df.id) == set(bigru_fasttext_preds.id)
assert set(train_df.id) == set(bigru_elmo_preds.id)


In [21]:
from sklearn.metrics import roc_auc_score
print('BERT ROC-AUC: {0:.4f}'.format(roc_auc_score(train_df[TARGET_COLS].values, bert_preds[TARGET_COLS].values)))
print('GPT ROC-AUC: {0:.4f}'.format(roc_auc_score(train_df[TARGET_COLS].values, gpt_preds[TARGET_COLS].values)))
print('FT BiGRU ROC-AUC: {0:.4f}'.format(roc_auc_score(train_df[TARGET_COLS].values, bigru_fasttext_preds[TARGET_COLS].values)))
print('ELMo BiGRU ROC-AUC: {0:.4f}'.format(roc_auc_score(train_df[TARGET_COLS].values, bigru_elmo_preds[TARGET_COLS].values)))


BERT ROC-AUC: 0.9894
GPT ROC-AUC: 0.9880
FT BiGRU ROC-AUC: 0.9858
ELMo BiGRU ROC-AUC: 0.9867


In [22]:
import numpy as np
from scipy.stats.mstats import gmean
simple_avg = np.mean(np.stack([bert_preds[TARGET_COLS].values,
                               gpt_preds[TARGET_COLS].values,
                               bigru_fasttext_preds[TARGET_COLS].values,
                               bigru_elmo_preds[TARGET_COLS].values]), 
                     axis=0)
geo_avg = gmean(np.stack([bert_preds[TARGET_COLS].values,
                               gpt_preds[TARGET_COLS].values,
                               bigru_fasttext_preds[TARGET_COLS].values,
                               bigru_elmo_preds[TARGET_COLS].values]),
                axis=0)

print('Average ensembler ROC-AUC: {0:.4f}'.format(roc_auc_score(train_df[TARGET_COLS].values,
                                                                simple_avg)))
print('Geo mean ensembler ROC-AUC: {0:.4f}'.format(roc_auc_score(train_df[TARGET_COLS].values,
                                                                geo_avg)))

soft_col_labels = list(map(lambda x: 'soft_'+x, TARGET_COLS))
avg_df = pd.DataFrame(simple_avg, columns=soft_col_labels)
# pd.concat([train_df.reset_index(drop=True), avg_df.reset_index(drop=True)], axis=1).to_csv('../data/train_with_soft_targets.csv',index=False)



Average ensembler ROC-AUC: 0.9921
Geo mean ensembler ROC-AUC: 0.9917


In [26]:
bert_hardsoft_df = pd.read_csv('../data/preds_bert_hardsoft_targets.csv').sort_values(by='id')
print('BERT hard + soft ROC-AUC: {0:.4f}'.format(roc_auc_score(train_df[TARGET_COLS].values, bert_hardsoft_df[TARGET_COLS].values)))


BERT hard + soft ROC-AUC: 0.9702
